In [1]:
!nvidia-smi

Wed Sep 29 20:45:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:09:00.0 Off |                  Off |
| 71%   81C    P2   109W / 300W |  11643MiB / 48682MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wandb login

wandb: Currently logged in as: arutema47 (use `wandb login --relogin` to force relogin)


In [3]:
import argparse
import os
import shutil
import time
import pandas as pd
import matplotlib.pyplot as plt
import network.resnet_orig as resnet

import time
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

model_names = sorted(name for name in resnet.__dict__
    if name.islower() and not name.startswith("__")
                     and name.startswith("resnet")
                     and callable(resnet.__dict__[name]))

DATA_DIR = "train"

print_freq = 50

K = 4

In [4]:
train_df = pd.read_csv("trainLabels.csv")
train_df.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['label_enc'] = le.fit_transform(train_df['label'])
train_df.head()

,id,label,label_enc
0,1,frog,6
1,2,truck,9
2,3,truck,9
3,4,deer,4
4,5,automobile,1


In [6]:
# 5-flod
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_df["fold"] = -1
for i, (train_index, test_index) in enumerate(skf.split(train_df.id, train_df.label_enc)):
    train_df.loc[test_index, 'fold'] = i
train_df.head()

,id,label,label_enc,fold
0,1,frog,6,1
1,2,truck,9,1
2,3,truck,9,2
3,4,deer,4,1
4,5,automobile,1,2


# prepare dataset

In [7]:
mixup = False

class cifarDataset(Dataset):
    def __init__(self,
                 df,
                 rand=False,
                 transform=None,
                 test=False
                ):

        self.df = df.reset_index(drop=True)
        self.rand = rand
        self.transform = transform
        self.test = test

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.id
        
        images = cv2.imread(os.path.join(DATA_DIR, str(img_id)+".png"))
        
        # Load labels
        label = row.label_enc
        
        # aug
        if self.transform is not None:
            images = self.transform(image=images)['image']
        
        # Mixup part
        """
        rd = torch.rand(1)
        label2 = label
        gamma = np.array(np.ones(1)).astype(np.float32)[0]
        if mixup and rd < 0.3 and self.transform is not None and not self.test:
            mix_idx = np.random.random_integers(0, len(self.df))
            row2 = self.df.iloc[mix_idx]
            img_id2 = row2.id
            images2 = cv2.imread(os.path.join(DATA_DIR, str(img_id2)+".png"))
            
            if self.transform is not None:
                images2 = self.transform(image=images2)['image']
            
            # blend image
            gamma = np.array(np.random.beta(1,1)).astype(np.float32)
            images = ((images*gamma + images2*(1-gamma))).astype(np.uint8)
            # blend labels
            label2 = row2.label_enc
        """
              
        #images = images.astype(np.float32)
        #images /= 255
        images = images.transpose(2, 0, 1)
        
        label = label.astype(np.float32)
        #label2 = label2.astype(np.float32)
        return torch.tensor(images), torch.tensor(label),

In [8]:
import albumentations as A
import albumentations

imsize = 32
transforms_train = albumentations.Compose([
    albumentations.ShiftScaleRotate(scale_limit=0.3, rotate_limit=180,p=0.25),
    A.Cutout(num_holes=12, max_h_size=4, max_w_size=4, fill_value=0, p=0.25),
    #albumentations.Rotate(p=0.5),
    #albumentations.Transpose(p=0.5),
    #albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),   
    albumentations.RandomCrop(imsize, imsize, p=1.0), 
    albumentations.Normalize(mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225), p=1),
])

transforms_val = albumentations.Compose([albumentations.Resize(imsize, imsize, p=1.0),
                                         albumentations.Normalize(mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225), p=1),])

/home/ken/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:690: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


show imgs

In [9]:
dataset_show = cifarDataset(train_df, transform=transforms_train)
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
for i in range(3):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = np.random.randint(0, len(dataset_show))
        img, label = dataset_show[idx]
        img = img.flip(0) #BGR2RGB
        axarr[p].imshow(img.transpose(0,1).transpose(1,2))
        axarr[p].set_title(str(label))

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

# Trainer

In [10]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [11]:
def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    lambda_alpha = 0.0002
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)
        target2 = target

        target = target.long().cuda()
        target2 = target2.long().cuda()
        input_var = input.cuda()
        target_var = target

        if not mixup:
          alpha = 0
        else:
          alpha = 1
        

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # L2 regularization
        l2_alpha = 0.0
        for name, param in model.named_parameters():
            if "alpha" in name:
                l2_alpha += torch.pow(param, 2)
        loss += lambda_alpha * l2_alpha

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
    return losses.avg

def validate(val_loader, model, criterion):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.long().cuda()
            input_var = input.cuda()
            target_var = target.cuda()

            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1 = accuracy(output.data, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(val_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))
    for name, param in model.named_parameters():
        if "alpha" in name:
            print(name, param.item())
    return top1.avg, losses.avg

def save_checkpoint(state, filename='checkpoint.pth'):
    """
    Save the training model
    """
    torch.save(state, filename)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

# Train loop

In [12]:
train_dataset = cifarDataset(train_df[train_df.fold!=0], transform=transforms_train)
val_dataset = cifarDataset(train_df[train_df.fold==0], transform=transforms_val, test=True)

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=128, shuffle=True,
        num_workers=4, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=512, shuffle=False,
    num_workers=4, pin_memory=True)

In [13]:
for K in range(3,8):
    model = resnet.__dict__["resnet20"](K)
    model = model.cuda()

    import wandb
    watermark = "resnet20_paper_quant{}".format(K)
    wandb.init(project="pact",
                name=watermark)

    # define loss function (criterion) and pptimizer
    mixup = False
    criterion = nn.CrossEntropyLoss().cuda()
    def mixup_criterion(criterion, pred, y_a, y_b, lam):
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

    #optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-1,
                                momentum=0.9,
                                weight_decay=0.0002)

    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
                                                        milestones=[80, 150],)

    best_prec1 = 0
    os.makedirs("models", exist_ok=True)

    for epoch in range(200):
        # train for one epoch
        print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
        tloss = train(train_loader, model, criterion, optimizer, epoch)
        lr_scheduler.step()

        # evaluate on validation set
        prec1, valloss = validate(val_loader, model, criterion)

        # wandb
        wandb.log({'epoch': epoch, "prec":prec1, "train_loss": tloss, 'val_loss': valloss, "lr": optimizer.param_groups[0]["lr"],})

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)

        print("Best prec1 : ", best_prec1)
        if is_best:
            torch.save(model.state_dict(), os.path.join(f'models/resnet20_paper_quant{K}.pth'))

bit width: 3


wandb: Currently logged in as: arutema47 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


current lr 1.00000e-01
Epoch: [0][0/313]	Time 0.680 (0.680)	Data 0.154 (0.154)	Loss 7.6419 (7.6419)	Prec@1 7.812 (7.812)
Epoch: [0][50/313]	Time 0.063 (0.076)	Data 0.000 (0.003)	Loss 2.5179 (13.8807)	Prec@1 13.281 (11.091)
Epoch: [0][100/313]	Time 0.063 (0.069)	Data 0.000 (0.002)	Loss 2.5170 (8.2726)	Prec@1 17.969 (12.539)
Epoch: [0][150/313]	Time 0.063 (0.068)	Data 0.000 (0.001)	Loss 2.4125 (6.3605)	Prec@1 17.188 (13.680)
Epoch: [0][200/313]	Time 0.064 (0.067)	Data 0.000 (0.001)	Loss 2.3817 (5.3871)	Prec@1 20.312 (14.509)
Epoch: [0][250/313]	Time 0.064 (0.066)	Data 0.000 (0.001)	Loss 2.4851 (4.7933)	Prec@1 14.844 (15.326)
Epoch: [0][300/313]	Time 0.063 (0.066)	Data 0.000 (0.001)	Loss 2.3333 (4.3882)	Prec@1 32.812 (16.276)
Test: [0/20]	Time 0.282 (0.282)	Loss 2.0648 (2.0648)	Prec@1 21.289 (21.289)
 * Prec@1 20.690
alpha1 8.324317932128906
layer1.0.alpha1 8.324336051940918
layer1.0.alpha2 8.313596725463867
layer1.1.alpha1 8.32434368133545
layer1.1.alpha2 8.163472175598145
layer2.0.alpha

Epoch: [6][0/313]	Time 0.230 (0.230)	Data 0.164 (0.164)	Loss 1.5490 (1.5490)	Prec@1 45.312 (45.312)
Epoch: [6][50/313]	Time 0.064 (0.067)	Data 0.000 (0.003)	Loss 1.5559 (1.5614)	Prec@1 41.406 (44.378)
Epoch: [6][100/313]	Time 0.064 (0.066)	Data 0.000 (0.002)	Loss 1.4484 (1.5570)	Prec@1 44.531 (44.214)
Epoch: [6][150/313]	Time 0.063 (0.065)	Data 0.000 (0.001)	Loss 1.4204 (1.5441)	Prec@1 47.656 (44.800)
Epoch: [6][200/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 1.4971 (1.5331)	Prec@1 43.750 (45.208)
Epoch: [6][250/313]	Time 0.063 (0.064)	Data 0.000 (0.001)	Loss 1.4182 (1.5245)	Prec@1 45.312 (45.477)
Epoch: [6][300/313]	Time 0.064 (0.064)	Data 0.000 (0.001)	Loss 1.5748 (1.5141)	Prec@1 52.344 (45.855)
Test: [0/20]	Time 0.288 (0.288)	Loss 1.2951 (1.2951)	Prec@1 52.539 (52.539)
 * Prec@1 51.910
alpha1 2.6199843883514404
layer1.0.alpha1 2.6515064239501953
layer1.0.alpha2 2.7493338584899902
layer1.1.alpha1 2.675130605697632
layer1.1.alpha2 2.459699869155884
layer2.0.alpha1 2.69876360893249

Epoch: [12][0/313]	Time 0.223 (0.223)	Data 0.157 (0.157)	Loss 0.9380 (0.9380)	Prec@1 68.750 (68.750)
Epoch: [12][50/313]	Time 0.064 (0.067)	Data 0.000 (0.003)	Loss 1.0353 (1.0800)	Prec@1 62.500 (62.515)
Epoch: [12][100/313]	Time 0.064 (0.066)	Data 0.000 (0.002)	Loss 1.0983 (1.0841)	Prec@1 64.062 (62.160)
Epoch: [12][150/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.9383 (1.0783)	Prec@1 71.094 (62.567)
Epoch: [12][200/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.9182 (1.0733)	Prec@1 62.500 (62.729)
Epoch: [12][250/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 1.1681 (1.0734)	Prec@1 61.719 (62.609)
Epoch: [12][300/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 1.1283 (1.0707)	Prec@1 54.688 (62.731)
Test: [0/20]	Time 0.271 (0.271)	Loss 0.9213 (0.9213)	Prec@1 66.992 (66.992)
 * Prec@1 66.790
alpha1 1.9391868114471436
layer1.0.alpha1 1.5461310148239136
layer1.0.alpha2 1.839910626411438
layer1.1.alpha1 1.3620824813842773
layer1.1.alpha2 1.5869815349578857
layer2.0.alpha1 1.405866

...

Epoch: [120][0/313]	Time 0.216 (0.216)	Data 0.149 (0.149)	Loss 0.0689 (0.0689)	Prec@1 98.438 (98.438)
Epoch: [120][50/313]	Time 0.064 (0.066)	Data 0.000 (0.003)	Loss 0.1334 (0.1518)	Prec@1 94.531 (95.205)
Epoch: [120][100/313]	Time 0.063 (0.065)	Data 0.000 (0.002)	Loss 0.1842 (0.1515)	Prec@1 92.969 (95.150)
Epoch: [120][150/313]	Time 0.063 (0.064)	Data 0.000 (0.001)	Loss 0.1273 (0.1535)	Prec@1 94.531 (94.961)
Epoch: [120][200/313]	Time 0.063 (0.064)	Data 0.000 (0.001)	Loss 0.1313 (0.1506)	Prec@1 96.094 (95.025)
Epoch: [120][250/313]	Time 0.063 (0.064)	Data 0.000 (0.001)	Loss 0.1299 (0.1509)	Prec@1 94.531 (95.032)
Epoch: [120][300/313]	Time 0.063 (0.064)	Data 0.000 (0.001)	Loss 0.2148 (0.1501)	Prec@1 92.188 (95.087)
Test: [0/20]	Time 0.284 (0.284)	Loss 0.1967 (0.1967)	Prec@1 94.336 (94.336)
 * Prec@1 92.490
alpha1 1.162499189376831
layer1.0.alpha1 0.7788605690002441
layer1.0.alpha2 1.1066889762878418
layer1.1.alpha1 0.6630980968475342
layer1.1.alpha2 1.0834914445877075
layer2.0.alpha1 0

Epoch: [126][0/313]	Time 0.222 (0.222)	Data 0.153 (0.153)	Loss 0.1188 (0.1188)	Prec@1 95.312 (95.312)
Epoch: [126][50/313]	Time 0.065 (0.067)	Data 0.000 (0.003)	Loss 0.1612 (0.1486)	Prec@1 94.531 (94.991)
Epoch: [126][100/313]	Time 0.063 (0.065)	Data 0.000 (0.002)	Loss 0.0994 (0.1531)	Prec@1 98.438 (94.926)
Epoch: [126][150/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.1753 (0.1504)	Prec@1 95.312 (94.971)
Epoch: [126][200/313]	Time 0.063 (0.065)	Data 0.000 (0.001)	Loss 0.0842 (0.1499)	Prec@1 96.875 (95.072)
Epoch: [126][250/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.1944 (0.1510)	Prec@1 93.750 (95.048)
Epoch: [126][300/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.0965 (0.1509)	Prec@1 96.875 (95.045)
Test: [0/20]	Time 0.284 (0.284)	Loss 0.1797 (0.1797)	Prec@1 94.531 (94.531)
 * Prec@1 92.270
alpha1 1.0849709510803223
layer1.0.alpha1 0.7231065630912781
layer1.0.alpha2 1.0446244478225708
layer1.1.alpha1 0.614225447177887
layer1.1.alpha2 1.0505894422531128
layer2.0.alpha1 0

Epoch: [132][0/313]	Time 0.215 (0.215)	Data 0.148 (0.148)	Loss 0.1984 (0.1984)	Prec@1 94.531 (94.531)
Epoch: [132][50/313]	Time 0.064 (0.067)	Data 0.000 (0.003)	Loss 0.1729 (0.1469)	Prec@1 95.312 (95.236)
Epoch: [132][100/313]	Time 0.064 (0.065)	Data 0.000 (0.002)	Loss 0.1201 (0.1453)	Prec@1 96.875 (95.181)
Epoch: [132][150/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.0815 (0.1436)	Prec@1 96.875 (95.266)
Epoch: [132][200/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.1195 (0.1428)	Prec@1 96.875 (95.208)
Epoch: [132][250/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.1269 (0.1442)	Prec@1 97.656 (95.210)
Epoch: [132][300/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.1466 (0.1457)	Prec@1 96.094 (95.183)
Test: [0/20]	Time 0.276 (0.276)	Loss 0.1767 (0.1767)	Prec@1 94.141 (94.141)
 * Prec@1 92.140
alpha1 1.082770824432373
layer1.0.alpha1 0.7136817574501038
layer1.0.alpha2 1.0130186080932617
layer1.1.alpha1 0.5905248522758484
layer1.1.alpha2 1.006473183631897
layer2.0.alpha1 0.

Epoch: [138][0/313]	Time 0.233 (0.233)	Data 0.163 (0.163)	Loss 0.1001 (0.1001)	Prec@1 96.094 (96.094)
Epoch: [138][50/313]	Time 0.063 (0.067)	Data 0.000 (0.003)	Loss 0.1601 (0.1478)	Prec@1 94.531 (94.853)
Epoch: [138][100/313]	Time 0.064 (0.066)	Data 0.000 (0.002)	Loss 0.1684 (0.1464)	Prec@1 94.531 (95.057)
Epoch: [138][150/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.1183 (0.1464)	Prec@1 95.312 (95.116)
Epoch: [138][200/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.1832 (0.1451)	Prec@1 95.312 (95.173)
Epoch: [138][250/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.1202 (0.1448)	Prec@1 96.094 (95.210)
Epoch: [138][300/313]	Time 0.064 (0.065)	Data 0.000 (0.001)	Loss 0.2624 (0.1443)	Prec@1 92.969 (95.232)
Test: [0/20]	Time 0.277 (0.277)	Loss 0.1701 (0.1701)	Prec@1 94.727 (94.727)
 * Prec@1 92.510
alpha1 1.0693154335021973
layer1.0.alpha1 0.6775432825088501
layer1.0.alpha2 0.9726573824882507
layer1.1.alpha1 0.5515686869621277
layer1.1.alpha2 1.0357176065444946
layer2.0.alpha1 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
arch = "resnet34"

for K in range(3,8):
    model = resnet.__dict__[arch](K)
    model = model.cuda()

    import wandb
    watermark = "{}_paper_quant{}".format(arch,K)
    wandb.init(project="pact",
                name=watermark)

    # define loss function (criterion) and pptimizer
    mixup = False
    criterion = nn.CrossEntropyLoss().cuda()
    def mixup_criterion(criterion, pred, y_a, y_b, lam):
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

    #optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-1,
                                momentum=0.9,
                                weight_decay=0.0002)

    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
                                                        milestones=[80, 150],)

    best_prec1 = 0
    os.makedirs("models", exist_ok=True)

    for epoch in range(200):
        # train for one epoch
        print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
        tloss = train(train_loader, model, criterion, optimizer, epoch)
        lr_scheduler.step()

        # evaluate on validation set
        prec1, valloss = validate(val_loader, model, criterion)

        # wandb
        wandb.log({'epoch': epoch, "prec":prec1, "train_loss": tloss, 'val_loss': valloss, "lr": optimizer.param_groups[0]["lr"],})

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)

        print("Best prec1 : ", best_prec1)
        if is_best:
            torch.save(model.state_dict(), os.path.join(f'models/{arch}_paper_quant{K}.pth'))

bit width: 3


epoch,199
prec,93.06
train_loss,0.10556
val_loss,0.23313
lr,0.001
_runtime,4326
_timestamp,1632937600
_step,199


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
prec,▁▄▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████████
train_loss,█▆▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▃▂▂▂▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,████████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


current lr 1.00000e-01
Epoch: [0][0/313]	Time 0.256 (0.256)	Data 0.148 (0.148)	Loss 9.5734 (9.5734)	Prec@1 9.375 (9.375)
Epoch: [0][50/313]	Time 0.106 (0.109)	Data 0.000 (0.003)	Loss 2.8731 (16.2324)	Prec@1 16.406 (11.734)
Epoch: [0][100/313]	Time 0.105 (0.107)	Data 0.000 (0.002)	Loss 2.7176 (9.6006)	Prec@1 21.094 (13.436)
Epoch: [0][150/313]	Time 0.105 (0.107)	Data 0.000 (0.001)	Loss 2.6285 (7.3266)	Prec@1 22.656 (15.066)
Epoch: [0][200/313]	Time 0.106 (0.106)	Data 0.000 (0.001)	Loss 2.6343 (6.1689)	Prec@1 20.312 (16.142)
Epoch: [0][250/313]	Time 0.107 (0.106)	Data 0.000 (0.001)	Loss 2.6166 (5.4592)	Prec@1 22.656 (17.013)
Epoch: [0][300/313]	Time 0.105 (0.106)	Data 0.000 (0.001)	Loss 2.5649 (4.9759)	Prec@1 10.938 (17.546)
Test: [0/20]	Time 0.334 (0.334)	Loss 2.0656 (2.0656)	Prec@1 22.070 (22.070)
 * Prec@1 23.070
alpha1 8.321855545043945
layer1.0.alpha1 8.326095581054688
layer1.0.alpha2 8.322935104370117
layer1.1.alpha1 8.324823379516602
layer1.1.alpha2 8.318604469299316
layer1.2.alph

...

current lr 1.00000e-03
Epoch: [160][0/313]	Time 0.272 (0.272)	Data 0.161 (0.161)	Loss 0.1741 (0.1741)	Prec@1 93.750 (93.750)
Epoch: [160][50/313]	Time 0.113 (0.115)	Data 0.000 (0.003)	Loss 0.0713 (0.1167)	Prec@1 97.656 (95.971)
Epoch: [160][100/313]	Time 0.113 (0.114)	Data 0.000 (0.002)	Loss 0.1612 (0.1222)	Prec@1 96.094 (95.846)
Epoch: [160][150/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.1795 (0.1163)	Prec@1 95.312 (96.068)
Epoch: [160][200/313]	Time 0.111 (0.113)	Data 0.000 (0.001)	Loss 0.1263 (0.1191)	Prec@1 97.656 (95.989)
Epoch: [160][250/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.1489 (0.1182)	Prec@1 96.094 (96.007)
Epoch: [160][300/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.0649 (0.1184)	Prec@1 98.438 (96.026)
Test: [0/20]	Time 0.325 (0.325)	Loss 0.1989 (0.1989)	Prec@1 95.117 (95.117)
 * Prec@1 92.670
alpha1 0.9977473616600037
layer1.0.alpha1 0.5482935309410095
layer1.0.alpha2 0.9295732975006104
layer1.1.alpha1 0.26660293340682983
layer1.1.alpha2 0.933772683

Epoch: [164][50/313]	Time 0.112 (0.116)	Data 0.000 (0.003)	Loss 0.1662 (0.1151)	Prec@1 95.312 (96.278)
Epoch: [164][100/313]	Time 0.112 (0.114)	Data 0.000 (0.002)	Loss 0.1719 (0.1166)	Prec@1 94.531 (96.078)
Epoch: [164][150/313]	Time 0.113 (0.113)	Data 0.000 (0.001)	Loss 0.0731 (0.1186)	Prec@1 96.875 (96.006)
Epoch: [164][200/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.1050 (0.1155)	Prec@1 96.094 (96.164)
Epoch: [164][250/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.0851 (0.1152)	Prec@1 96.875 (96.184)
Epoch: [164][300/313]	Time 0.111 (0.113)	Data 0.000 (0.001)	Loss 0.1331 (0.1162)	Prec@1 94.531 (96.135)
Test: [0/20]	Time 0.336 (0.336)	Loss 0.1864 (0.1864)	Prec@1 94.727 (94.727)
 * Prec@1 92.760
alpha1 0.9991459846496582
layer1.0.alpha1 0.5430333614349365
layer1.0.alpha2 0.9237130880355835
layer1.1.alpha1 0.26378604769706726
layer1.1.alpha2 0.9351170659065247
layer1.2.alpha1 0.6040881276130676
layer1.2.alpha2 0.9931049346923828
layer2.0.alpha1 0.793732762336731
layer2.0.alph

Epoch: [168][100/313]	Time 0.113 (0.114)	Data 0.000 (0.002)	Loss 0.1736 (0.1190)	Prec@1 92.969 (96.024)
Epoch: [168][150/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.0781 (0.1156)	Prec@1 96.094 (96.058)
Epoch: [168][200/313]	Time 0.113 (0.113)	Data 0.000 (0.001)	Loss 0.0667 (0.1181)	Prec@1 97.656 (96.020)
Epoch: [168][250/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.1244 (0.1171)	Prec@1 96.094 (96.025)
Epoch: [168][300/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.0950 (0.1155)	Prec@1 96.875 (96.078)
Test: [0/20]	Time 0.331 (0.331)	Loss 0.1931 (0.1931)	Prec@1 94.727 (94.727)
 * Prec@1 92.700
alpha1 0.9905424118041992
layer1.0.alpha1 0.5392095446586609
layer1.0.alpha2 0.9195411801338196
layer1.1.alpha1 0.26224735379219055
layer1.1.alpha2 0.9273332953453064
layer1.2.alpha1 0.6051778793334961
layer1.2.alpha2 0.990781843662262
layer2.0.alpha1 0.7904358506202698
layer2.0.alpha2 0.8781182765960693
layer2.1.alpha1 0.2958199381828308
layer2.1.alpha2 0.8327129483222961
layer2.2.al

Epoch: [172][200/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.1031 (0.1166)	Prec@1 96.875 (96.156)
Epoch: [172][250/313]	Time 0.112 (0.113)	Data 0.000 (0.001)	Loss 0.0936 (0.1146)	Prec@1 96.094 (96.231)
Epoch: [172][300/313]	Time 0.111 (0.113)	Data 0.000 (0.001)	Loss 0.0614 (0.1147)	Prec@1 96.875 (96.187)
Test: [0/20]	Time 0.321 (0.321)	Loss 0.1863 (0.1863)	Prec@1 95.312 (95.312)
 * Prec@1 92.690
alpha1 0.9921781420707703
layer1.0.alpha1 0.5375100374221802
layer1.0.alpha2 0.9162448048591614
layer1.1.alpha1 0.26041439175605774
layer1.1.alpha2 0.9240275621414185
layer1.2.alpha1 0.6004846096038818
layer1.2.alpha2 0.9896199107170105
layer2.0.alpha1 0.7848914265632629
layer2.0.alpha2 0.8698849081993103
layer2.1.alpha1 0.29622215032577515
layer2.1.alpha2 0.8221192359924316
layer2.2.alpha1 0.2731914222240448
layer2.2.alpha2 0.8436102271080017
layer2.3.alpha1 0.4327099323272705
layer2.3.alpha2 0.8907290101051331
layer3.0.alpha1 0.648765504360199
layer3.0.alpha2 0.8807600140571594
layer3.1

Epoch: [176][300/313]	Time 0.266 (0.160)	Data 0.000 (0.001)	Loss 0.0843 (0.1097)	Prec@1 98.438 (96.392)
Test: [0/20]	Time 0.495 (0.495)	Loss 0.1903 (0.1903)	Prec@1 95.117 (95.117)
 * Prec@1 92.730
alpha1 0.987460196018219
layer1.0.alpha1 0.5338073968887329
layer1.0.alpha2 0.9107036590576172
layer1.1.alpha1 0.25799912214279175
layer1.1.alpha2 0.9177005887031555
layer1.2.alpha1 0.5973081588745117
layer1.2.alpha2 0.9951046705245972
layer2.0.alpha1 0.7808435559272766
layer2.0.alpha2 0.8681884407997131
layer2.1.alpha1 0.2964380383491516
layer2.1.alpha2 0.8322550654411316
layer2.2.alpha1 0.2663118243217468
layer2.2.alpha2 0.8401656746864319
layer2.3.alpha1 0.4273487329483032
layer2.3.alpha2 0.8875317573547363
layer3.0.alpha1 0.6486174464225769
layer3.0.alpha2 0.8763926029205322
layer3.1.alpha1 0.30929872393608093
layer3.1.alpha2 0.8838664293289185
layer3.2.alpha1 0.28555670380592346
layer3.2.alpha2 0.8743069767951965
layer3.3.alpha1 0.26507481932640076
layer3.3.alpha2 0.8808720111846924
laye